In [1]:
# install ipydagred3

import ipydagred3
import spacy
from benepar.spacy_plugin import BeneparComponent

nlp = spacy.load("fr")
nlp.add_pipe(BeneparComponent('benepar_fr'))

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

tooltips = {
    "NP-SUJ": "Nom propre SUJET"
}

In [3]:
def processConstituency(pStr):
    nodes = []
    cur = "";
    stack = [];
    nid = 0;
    for i in range(len(pStr)):
        if(pStr[i] == ' ' or pStr[i] == '\n'):
            if (len(cur) > 0): 
                newNode = {
                    "nodeID": nid,
                    "nodeType": "Internal",
                    "name": cur,
                    "children": []
                }
                cur = "";
                nid += 1;
                if (len(stack) > 0):
                    stack[len(stack) - 1]["children"].append(newNode);
                stack.append(newNode);
                nodes.append(newNode)
        elif pStr[i] == ')':
            if (len(cur) > 0):
                newNode = {
                    "nodeID": nid,
                    "nodeType": "Leaf",
                    "name": cur,
                    "children": []
                }
                cur = "";
                nid += 1;
                stack[len(stack) - 1]["children"].append(newNode);
                nodes.append(newNode)
                stack.pop();
            else:
                if (len(stack) == 1):
                    root = stack[0]
                stack.pop();
        elif pStr[i] == '(':
            continue
        else:
            cur = cur + pStr[i];
    return nodes, root

def showDependencyGraph(nodes, parsed, sent):
    g = ipydagred3.Graph()
    wordIdx = 0
    for node in nodes:
        if len(node["children"]):
            tooltip = tooltips[node["name"]] if node["name"] in tooltips else node["name"]
        else:
#             tooltip = str(node["nodeID"])
            tooltip = f"WORD={sent[wordIdx].text} | DEP={sent[wordIdx].dep_} | POS={sent[wordIdx].pos_}\n"
            wordIdx += 1
        g.setNode(str(node["nodeID"]), label=node['name'], tooltip=tooltip, rx=5, ry=5, style="fill: " + ("white" if len(node["children"]) else "#00bcd4"));
#         g.setNode(str(node["nodeID"]), labelType="html", label=f"<span style='color: black; font-size: 12px;'>{node['name']}</span>", tooltip=tooltips[node["name"]] if node["name"] in tooltips else node["name"], rx=5, ry=5, style="fill: " + ("white" if len(node["children"]) else "#00bcd4"));

    def setEdge(parent):
        for i in range(len(parent["children"])):
            g.setEdge(str(parent["nodeID"]), str(parent["children"][i]["nodeID"]))
            setEdge(parent["children"][i])

    setEdge(parsed)
    widget = ipydagred3.DagreD3Widget(graph=g)
    display(widget)
    

In [7]:
import ipydagred3
import spacy
from benepar.spacy_plugin import BeneparComponent

nlp = spacy.load("fr")
nlp.add_pipe(BeneparComponent('benepar_fr'))

doc = nlp("Le gentil petit chat blanc mange du poisson.")
sent = list(doc.sents)[0]
nodes, parsed = processConstituency(sent._.parse_string)
showDependencyGraph(nodes, parsed, sent)


DagreD3Widget()